In [ ]:
# 如果你已經安裝過這些套件，可以略過這一格
%%bash
pip install --upgrade pip
pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo unsloth
pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
pip install fastapi nest_asyncio pyngrok uvicorn


  Using cached bitsandbytes-0.47.0-py3-none-manylinux_2_24_x86_64.whl.metadata (11 kB)
  Using cached xformers-0.0.29.post3-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (1.2 kB)
  Using cached trl-0.15.2-py3-none-any.whl.metadata (11 kB)
  Using cached cut_cross_entropy-25.1.1-py3-none-any.whl.metadata (9.3 kB)
  Using cached unsloth_zoo-2025.9.2-py3-none-any.whl.metadata (9.5 kB)
  Using cached unsloth-2025.9.1-py3-none-any.whl.metadata (52 kB)
Using cached xformers-0.0.29.post3-cp312-cp312-manylinux_2_28_x86_64.whl (43.4 MB)
Using cached trl-0.15.2-py3-none-any.whl (318 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 83.1 MB/s  0:00:00



In [ ]:
# STEP  掛載 Google Drive
################################################################################
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ngrok config add-authtoken 2xP3Ghx1IdbZs9tqf8unvgjaLTR_2ePx7XcAoEzvoUPgbkKzo
import nest_asyncio
nest_asyncio.apply()
from unsloth import FastLanguageModel
from peft import PeftModel

# 🧠 載入 Qwen3-14B（4bit 推論）
base, tok = FastLanguageModel.from_pretrained(
    "unsloth/Qwen3-14B",
    load_in_4bit=True,
    max_seq_length=2048,
)

# 🧠 載入你訓練的微調模型（LoRA Checkpoint）
model = PeftModel.from_pretrained(
    base,
    "/content/drive/MyDrive/sport/Qwen/trainer_output/checkpoint-870",
    is_trainable=False,
)

# 🔁 推論模式轉換
model = FastLanguageModel.for_inference(model)


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
==((====))==  Unsloth 2025.9.1: Fast Qwen3 patching. Transformers: 4.56.0.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
model = PeftModel.from_pretrained(
    base,
    "/content/drive/MyDrive/sport/Qwen/trainer_output/checkpoint-870",
    is_trainable=False,
)

# 🔁 推論模式轉換
model = FastLanguageModel.for_inference(model)

In [ ]:
from fastapi import FastAPI, Request
from fastapi.responses import StreamingResponse
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from transformers import TextIteratorStreamer
import threading, time

app = FastAPI()

# 🌐 CORS（允許手機前端連線）
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# 🧠 對話歷史紀錄（以 session_id 作為 key）
chat_histories = {}

# 🧾 前端請求格式
class ChatRequest(BaseModel):
    input: str

# 🚀 串流 API：支援多輪記憶
@app.post("/chat_stream")
async def chat_stream(req: ChatRequest, request: Request):
    session_id = request.query_params.get("session_id", "default")

    # 初始化對話歷史
    if session_id not in chat_histories:
        chat_histories[session_id] = []

    # 加入使用者訊息
    chat_histories[session_id].append({ "role": "user", "content": req.input })

    # 將歷史對話組合為 prompt
    prompt = tok.apply_chat_template(chat_histories[session_id], tokenize=False, add_generation_prompt=True)
    inputs = tok(prompt, return_tensors="pt").to("cuda")

    # 建立 streamer
    streamer = TextIteratorStreamer(tok, skip_prompt=True, skip_special_tokens=True)
    generation_kwargs = dict(inputs, streamer=streamer, max_new_tokens=1000)

    thread = threading.Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()

    # 初始化回應內容，逐字 append
    def token_generator():
        reply = ""
        for new_text in streamer:
            reply += new_text
            yield new_text
            time.sleep(0.01)
        # 將模型回應加入歷史
        chat_histories[session_id].append({ "role": "assistant", "content": reply })
        thread.join()

    return StreamingResponse(token_generator(), media_type="text/plain")


In [ ]:
from pyngrok import ngrok
import uvicorn

# 🔗 建立 ngrok 公開網址
public_url = ngrok.connect(8000)
print(f"🚀 你的 API 位址：{public_url}/chat_stream")

# 🚀 啟動 FastAPI 伺服器
uvicorn.run(app, port=8000)

🚀 你的 API 位址：NgrokTunnel: "https://87e2b9fa138a.ngrok-free.app" -> "http://localhost:8000"/chat_stream


INFO:     Started server process [2056]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     2402:7500:5fe:3d09::df3:9667:0 - "POST /chat_stream HTTP/1.1" 200 OK
INFO:     2402:7500:5fe:3d09::df3:9667:0 - "POST /chat_stream HTTP/1.1" 200 OK
INFO:     2402:7500:5fe:3d09::df3:9667:0 - "POST /chat_stream HTTP/1.1" 200 OK
INFO:     2402:7500:5fe:3d09::df3:9667:0 - "POST /chat_stream HTTP/1.1" 200 OK
INFO:     2402:7500:5fe:3d09::df3:9667:0 - "POST /chat_stream HTTP/1.1" 200 OK


Exception in thread Thread-17 (generate):
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.12/dist-packages/peft/peft_model.py", line 1973, in generate
    outputs = self.base_model.generate(*args, **kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/unsloth/models/llama.py", line 1799, in unsloth_fast_generate
    output = self._old_generate(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/_contextlib.py", line 120, in decorate_context
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/transformers/generation/utils.py", line 2539, in generate
    result = self._sample(
         

INFO:     2402:7500:5fe:3d09::df3:9667:0 - "POST /chat_stream HTTP/1.1" 500 Internal Server Error


...

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",

INFO:     2402:7500:5fe:3d09::df3:9667:0 - "POST /chat_stream HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",

INFO:     2402:7500:5fe:3d09::df3:9667:0 - "POST /chat_stream HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",

In [ ]:
# ④ 測試
from unsloth import FastLanguageModel
from peft import PeftModel
from transformers import TextStreamer
model = PeftModel.from_pretrained(
    base,
    "/content/drive/MyDrive/sport/Owen/trainer_output/checkpoint-644",
    is_trainable=False,
)
messages = [{"role": "user", "content": "設計訓練動作時，應以哪些肌肉群為主？"}]
prompt = tok.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)
inputs = tok(prompt, return_tensors="pt").to("cuda")

streamer = TextStreamer(tok, skip_prompt=True)
_ = model.generate(
    **inputs,
    max_new_tokens=1000,
    temperature=0.7,
    top_p=0.8,
    streamer=streamer,
)


<think>

</think>

應以主要肌肉群與全身重要肌群為主。<|im_end|>


In [ ]:
# ④ 測試
from unsloth import FastLanguageModel
from peft import PeftModel
from transformers import TextStreamer
model = PeftModel.from_pretrained(
    base,
    "/content/drive/MyDrive/sport/Owen/checkpoint-861",
    is_trainable=False,
)
messages = [{"role": "user", "content": "你是誰"}]
prompt = tok.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)
inputs = tok(prompt, return_tensors="pt").to("cuda")

streamer = TextStreamer(tok, skip_prompt=True)
_ = model.generate(
    **inputs,
    max_new_tokens=1000,
    temperature=1.6,
    top_p=0.8,
    streamer=streamer,
)


<think>

</think>

我是臺灣大學運動與 Leisure 科學系學生<|im_end|>


In [ ]:
# ④ 測試
from unsloth import FastLanguageModel
from peft import PeftModel
from transformers import TextStreamer
model = PeftModel.from_pretrained(
    base,
    "/content/drive/MyDrive/sport/Owen/checkpoint-400",
    is_trainable=False,
)
messages = [{"role": "user", "content": "你是誰"}]
prompt = tok.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)
inputs = tok(prompt, return_tensors="pt").to("cuda")

streamer = TextStreamer(tok, skip_prompt=True)
_ = model.generate(
    **inputs,
    max_new_tokens=1000,
    temperature=1.5,
    top_p=0.8,
    streamer=streamer,
)


<think>

</think>

我是一個大型語言模型，由阿里巴巴集團旗下的通義實驗室開發。我的中文名稱為通義千問，英文名稱為Qwen。我被設計用來回答問題、創作文字、邏輯推理，並在多個領域中提供幫助。<|im_end|>
